<a href="https://colab.research.google.com/github/Yaswanthyarra/GAN-for-microexpression-augmentation/blob/main/VGG_microexp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'
augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'

In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'

label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6, 'sadness': 5}

expression_images = []
expression_labels = []

for class_name, label in label_dict.items():
    class_dir = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224,224))
        if label==1:
          expression_images.append(image)
          expression_labels.append(label)
        if label==6:
          expression_images.append(image)
          expression_labels.append(label)



        expression_images.append(image)
        expression_labels.append(label)

augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'
label_dict = {'happiness': 0, 'repression': 2, 'surprise': 3, 'fear': 4, 'sadness': 5}

for class_name, label in label_dict.items():
    class_dir = os.path.join(augmented_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if label==0:
          expression_images.append(image)
          expression_labels.append(label)

        expression_images.append(image)
        expression_labels.append(label)

In [5]:
from PIL import Image
import numpy as np

# Assuming your expression_images array has shape (num_images, height, width, channels)

# Resize images to a consistent shape
target_shape = (224, 224)

resized_images = []
for img in expression_images:
    # Ensure the image has the correct shape and data type
    img = np.squeeze(img)  # Remove singleton dimensions
    img = img.astype('uint8')  # Convert to unsigned integer type
    img = Image.fromarray(img)

    # Resize the image
    img = img.resize(target_shape, Image.ANTIALIAS)
    resized_images.append(np.array(img))

expression_images = np.stack(resized_images)

# Ensure consistent data type
expression_images = expression_images.astype('float32')

# Rest of your code...


<ipython-input-5-aaf2c4bf8ca3>:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_shape, Image.ANTIALIAS)


In [6]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Convert lists to NumPy arrays
expression_images = np.array(expression_images)
expression_labels = np.array(expression_labels)

# Check the shape of the image data and labels
print("Expression Images Shape:", expression_images.shape)
print("Expression Labels Shape:", expression_labels.shape)

# Ensure consistent shape and data type
expression_images = np.array([img.astype('float32') for img in expression_images])
expression_images = np.stack(expression_images)  # Stack the images to create a 4D array

# Obtain the number of class labels
num_classes = len(np.unique(expression_labels))

# Convert labels to one-hot encoding
expression_labels_one_hot = to_categorical(expression_labels, num_classes=num_classes)

# Stratified Splitting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    expression_images, expression_labels_one_hot, test_size=0.2, stratify=expression_labels, random_state=42
)

# Load the VGG16 model pre-trained on ImageNet data
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Preprocess the images for the VGG model
train_images = preprocess_input(train_images)
test_images = preprocess_input(test_images)

# Create a sequential model
model = Sequential()

# Add the VGG base model
model.add(base_model)

# Flatten the output of the VGG model
model.add(Flatten())

# Add a dense layer with 256 units and ReLU activation
model.add(Dense(256, activation='relu'))

# Add the output layer with softmax activation for classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
model.summary()


Expression Images Shape: (782, 224, 224, 3)
Expression Labels Shape: (782,)
58889256/58889256 [==============================] - 0s 0us/step


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 21139271 (80.64 MB)
Trainable params: 21139271 (80.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model = tf.keras.models.load_model('/content/drive/My Drive/GAN_Models/VGG_Microexp.h5')
# Evaluate the model on the test set
from sklearn.metrics import classification_report


from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

# ... (previous code for model training and evaluation)

# Get predicted labels
predicted_probabilities = model.predict(test_images)
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Convert one-hot encoded test labels back to categorical labels
true_labels = np.argmax(test_labels, axis=1)

# Print accuracy, precision, recall, F1-score, and UAR
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Precision: {precision_score(true_labels, predicted_labels, average="weighted"):.4f}')
print(f'Recall: {recall_score(true_labels, predicted_labels, average="weighted"):.4f}')
print(f'F1-score: {f1_score(true_labels, predicted_labels, average="weighted"):.4f}')
print(f'Unweighted Average Recall (UAR): {balanced_accuracy_score(true_labels, predicted_labels):.4f}')


5/5 [==============================] - 92s 18s/step
Test Accuracy: 0.7707
Precision: 0.8076
Recall: 0.7707
F1-score: 0.7732
Unweighted Average Recall (UAR): 0.7721


In [ ]:

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

model.save('/content/drive/My Drive/GAN_Models/VGG_Microexp.h5')
model = tf.keras.models.load_model('/content/drive/My Drive/GAN_Models/VGG_Microexp.h5')


Epoch 1/10
16/16 [==============================] - 753s 47s/step - loss: 0.4651 - accuracy: 0.8460 - val_loss: 0.6419 - val_accuracy: 0.7840
Epoch 2/10
16/16 [==============================] - 756s 48s/step - loss: 0.4622 - accuracy: 0.8300 - val_loss: 0.7741 - val_accuracy: 0.7280
Epoch 3/10
16/16 [==============================] - 710s 44s/step - loss: 0.4799 - accuracy: 0.8340 - val_loss: 0.7196 - val_accuracy: 0.7920
Epoch 4/10
16/16 [==============================] - 710s 44s/step - loss: 0.4230 - accuracy: 0.8340 - val_loss: 0.6876 - val_accuracy: 0.7600
Epoch 5/10
16/16 [==============================] - 704s 44s/step - loss: 0.3322 - accuracy: 0.8960 - val_loss: 0.6326 - val_accuracy: 0.7840
Epoch 6/10
16/16 [==============================] - 704s 44s/step - loss: 0.3328 - accuracy: 0.8920 - val_loss: 0.8774 - val_accuracy: 0.7680
Epoch 7/10
16/16 [==============================] - 738s 46s/step - loss: 0.3412 - accuracy: 0.8820 - val_loss: 0.6696 - val_accuracy: 0.8000
Epoch 